In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [62]:
def process_DrugCount(drugcount):
    dc_map = {'1' : 1, '2':2, '3':3, '4':4, '5':5, '6':6, '7+' : 7}
    drugcount['DrugCount'] = drugcount.DrugCount.map(dc_map)
    drugcount['DrugCount'] = drugcount.DrugCount.astype(int)
    dc = drugcount.groupby(drugcount.Year, as_index=False)
    DrugCount_Y1 = dc.get_group('Y1')
    DrugCount_Y2 = dc.get_group('Y2')
    DrugCount_Y3 = dc.get_group('Y3')
    #print DrugCount_Y1
    #print DrugCount_Y2
    #print DrugCount_Y3
    return (DrugCount_Y1,DrugCount_Y2,DrugCount_Y3)

In [63]:
def replaceMonth(string):
    converted_string = ""
    map_of_format = {'0- 1 month' : "0_1", "1- 2 months": "1_2", "2- 3 months": "2_3", "3- 4 months": '3_4', "4- 5 months": "4_5", "5- 6 months": "5_6", "6- 7 months": "6_7", \
                   "7- 8 months" : "7_8", "8- 9 months": "8_9", "9-10 months": "9_10", "10-11 months": "10_11", "11-12 months": "11_12"}
    converted_string = string.map(map_of_format)

    return converted_string

In [64]:
def dummy_coding(df, variable,  dummy_na=False, drop_first = False, prefix=None):
    if prefix==None:
        prefix = variable
    outputdata = pd.get_dummies(df, columns=[variable], prefix= prefix, dummy_na=dummy_na, drop_first=drop_first)
    return outputdata

In [65]:
def process_yearly_DrugCount(aframe):
    processed_frame = None
    
    #print aframe.columns
    #print aframe
    aframe.drop("Year", axis = 1, inplace = True)
    after_month_conversion = aframe[['DSFS']].apply(replaceMonth)

    df = dummy_coding(after_month_conversion, "DSFS", dummy_na=True, drop_first=True) 
    joined =  pd.concat([aframe, df], axis = 1)
    joined.drop("DSFS", axis = 1, inplace = True)
    joined_grouped = joined.groupby("MemberID", as_index = False)
    joined_grouped_agg = joined_grouped.agg(np.sum)

    processed_frame = joined_grouped_agg.rename(columns={'DrugCount': 'Total_DrugCount'})
    #print processed_frame
    return processed_frame


In [66]:
def linear_regression(train_X, test_X, train_y, test_y):
    regr = linear_model.LinearRegression()
    regr.fit(train_X, train_y)
    #print 'Coefficients: \n', regr.coef_
    pred_y = regr.predict(test_X) 
    #print  (pred_y)
    #s=regr.score(test_X, pred_y)
    #print(s)
    import math
    from sklearn.metrics import r2_score
    from sklearn import metrics
    #pred_y=np.log(pred_y)
    #test_y=np.log(test_y)
    #mse1=np.log((pred_y - test_y) ** 2)
    #mse=np.mean(mse1)
    mse = np.mean( (pred_y - test_y) ** 2)
    #import math
    rmse = math.sqrt(mse)-1
    plt.show()
    print ("RMSE: %.5f" % rmse)
    #from sklearn.metrics import r2_score
    #print("Accuracy:",metrics.accuracy_score(test_y, pred_y))
    r2 = r2_score(pred_y, test_y)
    #print ("R2 value: %.2f" % r2)
    #from sklearn import svm
    #clf = svm.SVR()
    #clf.fit(train_X, train_y)
    #acc=clf.score(test_X, test_y)
    #print(acc)

In [67]:
def linear_svm(train_X, test_X, train_y, test_y):
    from sklearn import svm
    from sklearn import metrics
    import math
    clf = svm.SVC(kernel='rbf')
    clf.fit(train_X, train_y)
    y_pred = clf.predict(test_X)
    #acc=clf.score(test_X, test_y)
    #hello=2
    #print hello
    mse=np.mean((y_pred - test_y)**2)
    rmse=math.sqrt(mse)-1
    print ("RMSE: %.5f" % rmse)
    #print("Accuracy:",metrics.accuracy_score(test_y, y_pred))
    #print (acc)

In [68]:
def process_missing_numeric(df, variable):
    df[variable] = np.where(df[variable].isnull(),1,0)
    medianVar = df[variable].median()
    df[variable].fillna(medianVar, inplace= True)

In [69]:
def print_missing_variables(df):
    for variable in df.columns.tolist():
        percent = float(sum(df[variable].isnull()))/len(df.index)
        #print variable+":", percent

In [70]:
def main():
    pd.options.mode.chained_assignment = None 
    daysinhospital = pd.read_csv('DaysInHospital_Y2.csv') 
    drugcount = pd.read_csv('DrugCount.csv') 
    li = map(process_yearly_DrugCount, process_DrugCount(drugcount))
    DrugCount_Y1_New = li[0]
    #print DrugCount_Y1_New
    Master_Assn3 = None
    Master_Assn3 = pd.merge(daysinhospital, DrugCount_Y1_New, on='MemberID', how='left')
    process_missing_numeric(Master_Assn3, 'Total_DrugCount')
    Master_Assn3 = Master_Assn3.fillna(0)
    Master_Assn3.drop("MemberID", axis = 1, inplace = True)
    #print Master_Assn3.shape
    #print Master_Assn3.head(3)
    #print Master_Assn3
    dependent_var = 'DaysInHospital'
    independent_var = Master_Assn3.columns.tolist()
    independent_var.remove(dependent_var)
    print independent_var
    print dependent_var
    print Master_Assn3
    train_X, test_X, train_y, test_y= train_test_split(Master_Assn3[independent_var], Master_Assn3[dependent_var], test_size=0.3, random_state=123)
    #print train_X.shape, test_X.shape, train_y.shape, test_y.shape
    #print train_X
    linear_regression(train_X, test_X, train_y, test_y)
    linear_svm(train_X, test_X, train_y, test_y)

if __name__ == '__main__':
    main()

['ClaimsTruncated', 'Total_DrugCount', 'DSFS_10_11', 'DSFS_11_12', 'DSFS_1_2', 'DSFS_2_3', 'DSFS_3_4', 'DSFS_4_5', 'DSFS_5_6', 'DSFS_6_7', 'DSFS_7_8', 'DSFS_8_9', 'DSFS_9_10', 'DSFS_nan']
DaysInHospital
       ClaimsTruncated  DaysInHospital  Total_DrugCount  DSFS_10_11  \
0                    0               0                0         0.0   
1                    0               0                0         0.0   
2                    1               1                0         1.0   
3                    0               1                0         1.0   
4                    0               0                0         0.0   
5                    0               0                1         0.0   
6                    0               0                0         1.0   
7                    0               0                1         0.0   
8                    0               0                0         0.0   
9                    0               0                0         1.0   
10              

RMSE: 0.60024
RMSE: 0.69781
